In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#RULES
#Runs automated hands of blackjack using either an expert's strategy or our own
#Dealer hits on 16 and below, and a soft 17. Dealer stands on a hard 17 and up
#There is no splitting or doubling down in this implementation as we are comparing win chances
#using different strategies on hand values

In [3]:
#deals a single card
def dealCard():
    num = int(13 * np.random.rand() + 1)
    if num >= 10 :
        return 10
    elif num == 1 :
        return 11
    else :
        return num

In [4]:
dealCard()

10

In [5]:
#finds the total value of a hand
def handTotal(hand) :
    total = 0
    for num in hand :
        total += num
    return total

In [6]:
#determines if a player's hand busted
def didBust(hand) :
    return handTotal(hand) > 21

In [7]:
#changes the values of an ace if it causes the hand to bust
def changeAce(hand) :
    if didBust(hand) :
        for i, num in enumerate(hand) :
            if didBust(hand) :
                if hand[i] == 11 :
                    hand[i] = 1
    return hand

In [8]:
#deals a player's hand
def dealPlayer():
    hand = []
    hand.append(dealCard())
    hand.append(dealCard())
    hand = changeAce(hand)
    return hand

In [9]:
dealPlayer()

[11, 4]

In [10]:
hand = dealPlayer()
handTotal(hand)

15

In [11]:
hand

[5, 10]

In [12]:
#deals a card to a pre-existing hand
def dealOneCard(hand) :
    hand.append(dealCard())
    hand = changeAce(hand)
    return hand

In [13]:
dealOneCard(dealPlayer())

[9, 10, 6]

In [14]:
hand2 = dealOneCard(dealPlayer())
didBust(hand2)

True

In [15]:
hand2

[10, 10, 4]

In [16]:
ogHand = dealOneCard(dealPlayer())
ogHandCopy = []
for num in ogHand :
    ogHandCopy.append(num)
changeAce(ogHand)

[3, 6, 7]

In [17]:
ogHandCopy

[3, 6, 7]

In [18]:
#run's a dealer's hand
def playDealer(dealerHand) :
    while (handTotal(dealerHand) < 17) | ((handTotal(dealerHand) == 17) & hasAce11(dealerHand)) :
        dealerHand = dealOneCard(dealerHand)
    return dealerHand

In [56]:
#starts and runs a game n times using the given player strategy
def startGame(n, playerStrat) :
    results_df = pd.DataFrame(columns = ['PlayerHand', 'DealerHand', 'Winner', 'Result', 'Doubled'])
    for i in range(n) :
        winner = ''
        result = 0
        player = dealPlayer()
        dealer = dealPlayer()
        doubleDown = 1
        if handTotal(player) != 21 :
            if handTotal(dealer) == 21 :
                winner = "Dealer"
                result = -1
            else :
                if playerStrat == 'Ours' :
                    player, doubleDown = runHandOurs(player, dealer)
                elif playerStrat == 'Expert' :
                    player, doubleDown = runHandExpert(player, dealer)
                elif playerStrat == 'Builder' :
                    player, doubleDown = runHandBuildStrat(player, dealer)
                else :
                    player, doubleDown = runHandOurs(player, dealer)
                if didBust(player) == False :
                    dealer = playDealer(dealer)
                    if didBust(dealer) | (handTotal(player) > handTotal(dealer)) :
                        winner = 'Player'
                        result = 1 * doubleDown
                    elif handTotal(player) == handTotal(dealer) :
                        winner = 'Push'
                        result = 0
                    else :
                        winner = "Dealer"
                        result = -1 * doubleDown
                else :
                    winner = 'Dealer'
                    result = -1 * doubleDown
        else :
            winner = 'Player'
            result = 1.5
        results_df = results_df.append({'PlayerHand' : player, 'DealerHand' : dealer, 'Winner' : winner, 'Result' : result, 'Doubled' : doubleDown != 1}, 
                                       ignore_index = True)
    return results_df

In [57]:
len([0, 5])

2

In [58]:
7 in range(5, 7)

False

In [59]:
def runHandExpert(player, dealer) :
    dealerShows = dealer[0]
    stand = False
    double = 1
    while (didBust(player) == False) & (stand == False) :
        curTotal = handTotal(player)
        if curTotal <= 11 :
            if len(player) == 2 :
                if dealerShows in range(3, 7) :
                    double = 2
                    player = dealOneCard(player)
                    return player, double
                elif dealerShows in range(2, 10) :
                    if curTotal in range(10, 12) :
                        double = 2
                        player = dealOneCard(player)
                        return player, double
                elif dealerShows == 10 :
                    if curTotal == 11 :
                        double = 2
                        player = dealOneCard(player)
                        return player, double
            player = dealOneCard(player)
        elif curTotal == 12 :
            if hasAce11(player) :
                player = dealOneCard(player)
            else :
                if dealerShows in range(4, 7) :
                    stand = True
                else :
                    player = dealOneCard(player)
        elif curTotal in range(13, 17) :
            if hasAce11(player) :
                if len(player) == 2 :
                    if dealerShows in range(5, 7) :
                        double = 2
                        player = dealOneCard(player)
                        return player, double
                    elif dealerShows == 4 :
                        if curTotal in range(15, 17) :
                            double = 2
                            player = dealOneCard(player)
                            return player, double
                player = dealOneCard(player)
            else :
                if dealerShows in range(2, 7) :
                    stand = True
                else :
                    player = dealOneCard(player)
        elif curTotal == 17 :
            if hasAce11(player) :
                if len(player) == 2 & dealerShows in range(3, 7) :
                    double = 2
                    player = dealOneCard(player)
                    return player, double
                player = dealOneCard(player)
            else :
                stand = True
        elif curTotal == 18 :
            if hasAce11(player) :
                if dealerShows in range(9, 12) :
                    player = dealOneCard(player)
                else :
                    if len(player) == 2 & dealerShows in range(3, 7) :
                        double = 2
                        player = dealOneCard(player)
                        return player, double
                    stand = True
            else :
                stand = True
        else :
            stand = True
    return player, double

In [202]:
def runHandOurs(player, dealer) :
    dealerShows = dealer[0]
    stand = False
    double = 1
    while (didBust(player) == False) & (stand == False) :
        curTotal = handTotal(player)
        if curTotal <= 11 :
            player = dealOneCard(player)
        elif curTotal == 12 :
            if hasAce11(player) :
                player = dealOneCard(player)
            else :
                if dealerShows in [2, 3] :
                    stand = True
                else :
                    player = dealOneCard(player)
        elif curTotal in range(13, 16) :
            if hasAce11(player) :
                player = dealOneCard(player)
            else :
                if dealerShows in range((curTotal - 10), 8) :
                    stand = True
                else :
                    player = dealOneCard(player)
        elif curTotal == 16 :
            if hasAce11(player) :
                player = dealOneCard(player)
            else :
                if dealerShows in range(2, 10) :
                    stand = True
                else :
                    player = dealOneCard(player)
        else :
            stand = True
    return player, double

In [220]:
def runHandBuildStrat(player, dealer) :
    dealerShows = dealer[0]
    stand = False
    double = 1
    while (didBust(player) == False) & (stand == False) :
        curTotal = handTotal(player)
        if curTotal == 8 :
            player = dealOneCard(player)
        elif curTotal == 9 :
            if len(player) == 2 & dealerShows in range(3, 6) :
                double = 2
                player = dealOneCard(player)
                return player, double
            player = dealOneCard(player)
        elif curTotal in [10, 11] :
            if (len(player) == 2) & (dealerShows != 10) :
                double = 2
                player = dealOneCard(player)
                return player, double
            player = dealOneCard(player)
        elif curTotal == 12 :
            if hasAce11(player) :
                if dealerShows == 6 :
                    if len(player) == 2 :
                        double = 2
                        player = dealOneCard(player)
                        return player, double
                player = dealOneCard(player)
            else :
                if dealerShows in range(4, 7):
                    stand = True
                else :
                    player = dealOneCard(player)
        elif curTotal == 13 :
            if hasAce11(player) :
                if dealerShows == 6 :
                    if len(player) == 2 :
                        double = 2
                        player = dealOneCard(player)
                        return player, double
                player = dealOneCard(player)
            else :
                if dealerShows in range(2, 7):
                    stand = True
                else :
                    player = dealOneCard(player)
        elif curTotal == 14 :
            if hasAce11(player) :
                if dealerShows in [5, 6] :
                    if len(player) == 2 :
                        double = 2
                        player = dealOneCard(player)
                        return player, double
                player = dealOneCard(player)
            else :
                if (dealerShows in range(2, 7)) | (dealerShows == 11) :
                    stand = True
                else :
                    player = dealOneCard(player)
        elif curTotal == 15 :
            if hasAce11(player) :
                if dealerShows in [4, 5, 6] :
                    if len(player) == 2 :
                        double = 2
                        player = dealOneCard(player)
                        return player, double
                player = dealOneCard(player)
            else :
                if (dealerShows in range(2, 7)) | (dealerShows == 11) :
                    stand = True
                else :
                    player = dealOneCard(player)
        elif curTotal == 16 :
            if hasAce11(player) :
                if dealerShows in [4, 5, 6] :
                    if len(player) == 2 :
                        double = 2
                        player = dealOneCard(player)
                        return player, double
                player = dealOneCard(player)
            else :
                if (dealerShows in range(2, 7)) | (dealerShows in range(10, 12)) :
                    stand = True
                else :
                    player = dealOneCard(player)
        elif curTotal == 17 :
            if hasAce11(player) :
                if dealerShows in [3, 4, 5, 6] :
                    if len(player) == 2 :
                        double = 2
                        player = dealOneCard(player)
                        return player, double
                player = dealOneCard(player)
            else :
                stand = True
        elif curTotal == 18 :
            if hasAce11(player) :
                if dealerShows in [3, 4, 5, 6] :
                    if len(player) == 2 :
                        double = 2
                        player = dealOneCard(player)
                        return player, double
                if dealerShows in range(2, 9):
                    stand = True
                else :
                    player = dealOneCard(player)
            else :
                stand = True
        elif curTotal >= 19 :
            stand = True
        else :
            stand = True
    return player, double

In [23]:
def hasAce11(hand) :
    for num in hand :
        if num == 11 :
            return True
    return False

In [50]:
player, double = runHandExpert([5, 6], [6, 10])
double

2

In [224]:
results = startGame(100, 'Expert')
#results['Result'].sum() --- returns the total
results.Winner.value_counts(normalize=True)

Dealer    0.49
Player    0.42
Push      0.09
Name: Winner, dtype: float64

In [225]:
results.Result.sum()

-0.5

In [226]:
results.head(10)

,PlayerHand,DealerHand,Winner,Result,Doubled
0,"[8, 10]","[10, 6, 10]",Player,1,False
1,"[5, 10, 4]","[10, 6, 5]",Dealer,-1,False
2,"[9, 3, 8]","[1, 2, 10, 6]",Player,1,False
3,"[10, 11]","[3, 4]",Player,1.5,False
4,"[9, 6]","[5, 2, 10]",Dealer,-1,False
5,"[10, 11]","[4, 10]",Player,1.5,False
6,"[8, 10]","[4, 3, 8, 10]",Player,1,False
7,"[10, 10]","[10, 10]",Push,0,False
8,"[8, 10]","[1, 3, 10, 2, 3]",Dealer,-1,False
9,"[9, 10]","[10, 2, 9]",Dealer,-1,False


In [227]:
results = startGame(100, 'Builder')
#results['Result'].sum() --- returns the total
results.Winner.value_counts(normalize=True)

Player    0.48
Dealer    0.48
Push      0.04
Name: Winner, dtype: float64

In [273]:
results = startGame(100, 'Builder')
results['Result'].sum()

6.5

In [240]:
# compares the builder strategy to the expert strategy by running 'numGames' simulations, 'numComps' times
def compareStrats(numComps, numGames) :
    betterStrat = pd.DataFrame(columns = ['BetterStrat', 'OursPositive?', 'ExpertPositive?', 'Us - Expert'])
    for i in range(numComps) :
        winner = 'Error'
        ourResult = startGame(numGames, 'Builder')
        ourNormal = ourResult.Result.sum()
        expResult = startGame(numGames, 'Expert')
        expNormal = expResult.Result.sum()
        ourDif = ourNormal
        expDif = expNormal
        if ourDif > expDif :
            winner = 'Ours'
        elif ourDif == expDif :
            winner = 'Push'
        elif ourDif < expDif :
            winner = 'Expert'
        else :
            winner = 'Error'
        betterStrat = betterStrat.append({'BetterStrat' : winner, 'OursPositive?' : ourDif > 0, 'ExpertPositive?' : expDif > 0, 'Us - Expert' : float(format(ourDif - expDif, '.4f'))}, ignore_index = True)
    return betterStrat

In [279]:
# shows how each simulation played out
answers = compareStrats(100, 100)
answers.head(10)

,BetterStrat,OursPositive?,ExpertPositive?,Us - Expert
0,Ours,True,False,13.5
1,Ours,False,False,19.0
2,Expert,False,False,-2.5
3,Ours,False,False,12.0
4,Ours,True,True,8.0
5,Expert,False,False,-5.0
6,Expert,True,True,-5.0
7,Ours,True,True,3.5
8,Expert,False,False,-5.0
9,Expert,True,True,-12.0


In [280]:
# Distribution of which strategy is best
answers.BetterStrat.value_counts(normalize=True)

Ours      0.58
Expert    0.42
Name: BetterStrat, dtype: float64

In [281]:
# average number of more money we win compared to the expert
# positive - we won more money than the experts did
# negative - expert won more money than we did 
answers['Us - Expert'].mean()

2.465

In [282]:
# Of all simulations (representing a night at the casino), how many more nights did we walk away with money than
#  the experts would have
# In order to walk away with money, the number of hands won compared to the dealer must be positive
answers['OursPositive?'].value_counts()[True] - answers['ExpertPositive?'].value_counts()[True]

10

In [283]:
# How likely are we to walk away with money using our strategy
answers['OursPositive?'].value_counts()[True] / len(answers['OursPositive?'])

0.49

In [251]:
differences = pd.DataFrame([startGame(84, 'Builder').Winner.value_counts(normalize=True) for i in range(1000)])
differences['Difference'] = differences['Player'] - differences['Dealer']
gt0 = differences.loc[differences['Difference'] > 0]
e0 = differences.loc[differences['Difference'] == 0]
percent = len(gt0) / (len(differences) - len(e0))
hist = differences.Difference.hist(histtype='step', bins = 12)
percent, hist, differences.Difference.mean()

KeyboardInterrupt: 

In [ ]:
differences = pd.DataFrame([startGame(84, 'Expert').Winner.value_counts(normalize=True) for i in range(100)])
differences['Difference'] = differences['Player'] - differences['Dealer']
hist = differences.Difference.hist(histtype='step', bins = 12)
hist

In [198]:
# Finds the players chances of winning by either hitting or staying from having the given hand
# chances of winning are determined by running 'tests' amount of simulations using the player's hand total and
#  the dealer's show card
def testChances(total, tests) :
    table = pd.DataFrame(columns = {'2', '3', '4', '5', '6', '7', '8', '9', '10', '11'})
    options = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
    option2 = 0
    option3 = 0
    option4 = 0
    option5 = 0
    option6 = 0
    option7 = 0
    option8 = 0
    option9 = 0
    option10 = 0
    option11 = 0
    for n in options :
        numResults = pd.DataFrame(columns = {'HitWin%: ', 'StayWin%: '})
        hitWins = 0
        hitPushes = 0
        standWins = 0
        standPushes = 0
        for num in range(tests) :
            playerHand = [10, total - 10]
            card = dealCard()
            if n == 11 :
                while card == 10 :
                    card = dealCard()
            dealerHand = [n, card]
            if not (handTotal(dealerHand) == 21) :
                playerHand = dealOneCard(playerHand)
                playerHand, double = runHandBuildStrat(playerHand, dealerHand)
                if didBust(playerHand) == False :
                    dealerHand = playDealer(dealerHand)
                    if didBust(dealerHand) | (handTotal(playerHand) > handTotal(dealerHand)) :
                         hitWins += 1  
                    elif handTotal(playerHand) == handTotal(dealerHand) :
                         hitPushes += 1
        for num in range(tests) :
            playerHand = [10, total - 10]
            card = dealCard()
            if n == 11 :
                while card == 10 :
                    card = dealCard()
            dealerHand = [n, card]
            dealerHand = playDealer(dealerHand)
            if didBust(dealerHand) | (handTotal(playerHand) > handTotal(dealerHand)) :
                standWins += 1  
            elif handTotal(playerHand) == handTotal(dealerHand) :
                standPushes += 1
        ph = hitWins / (tests - hitPushes)
        eh = (2 * ph) - 1
        ps = standWins / (tests - standPushes)
        es = (2 * ps) - 1
        numResults = numResults.append({'HitWin%: ' : eh, 'StayWin%: ' : es}, ignore_index = True)
        if n == 2:
            option2 = numResults
        elif n == 3:
            option3 = numResults
        elif n == 4:
            option4 = numResults
        elif n == 5:
            option5 = numResults
        elif n == 6:
            option6 = numResults
        elif n == 7:
            option7 = numResults
        elif n == 8:
            option8 = numResults
        elif n == 9:
            option9 = numResults
        elif n == 10:
            option10 = numResults
        elif n == 11:
            option11 = numResults
    table = table.append({'2' : option2, '3' : option3, '4' : option4, '5' : option5, '6' : option6, '7' : option7, '8' : option8, '9' : option9, '10' : option10, '11' : option11}, ignore_index = True)
    table = table[['2', '3', '4', '5', '6', '7', '8', '9', '10', '11']]
    return table

In [153]:
testChances(12, 100000)

,2,3,4,5,6,7,8,9,10,11
0,HitWin%: StayWin%: 0 0.545351 0.35954,HitWin%: StayWin%: 0 0.557214 0.37664,HitWin%: StayWin%: 0 0.567912 0.39576,HitWin%: StayWin%: 0 0.583909 0.41887,HitWin%: StayWin%: 0 0.600743 0.44041,HitWin%: StayWin%: 0 0.59185 0.2623,HitWin%: StayWin%: 0 0.555428 0.24626,HitWin%: StayWin%: 0 0.501214 0.22791,HitWin%: StayWin%: 0 0.422742 0.21076,HitWin%: StayWin%: 0 0.501081 0.28095


In [38]:
# Regardless of the choice, a player is very unlikely to win if dealt a 12
testChances(12, 100000)

,2,3,4,5,6,7,8,9,10,11
0,HitWin%: StayWin%: 0 0.364599 0.36018,HitWin%: StayWin%: 0 0.378736 0.37524,HitWin%: StayWin%: 0 0.390566 0.399,HitWin%: StayWin%: 0 0.396758 0.41541,HitWin%: StayWin%: 0 0.409649 0.44287,HitWin%: StayWin%: 0 0.385353 0.26302,HitWin%: StayWin%: 0 0.351725 0.24682,HitWin%: StayWin%: 0 0.315605 0.22972,HitWin%: StayWin%: 0 0.267368 0.21226,HitWin%: StayWin%: 0 0.218541 0.19518


In [39]:
# No matter what the dealer shows, a player should always stay on 18
testChances(18, 100000)

,2,3,4,5,6,7,8,9,10,11
0,HitWin%: StayWin%: 0 0.180417 0.562998,HitWin%: StayWin%: 0 0.181284 0.581091,HitWin%: StayWin%: 0 0.180998 0.596586,HitWin%: StayWin%: 0 0.183297 0.613008,HitWin%: StayWin%: 0 0.18727 0.627732,HitWin%: StayWin%: 0 0.20079 0.733033,HitWin%: StayWin%: 0 0.197039 0.582852,HitWin%: StayWin%: 0 0.177573 0.395244,HitWin%: StayWin%: 0 0.148673 0.363191,HitWin%: StayWin%: 0 0.120053 0.286443


In [199]:
# Finds the players chances of winning by either hitting or staying from having the given hand
# chances of winning are determined by running 'tests' amount of simulations using the player's hand total and
#  the dealer's show card
def testChancesDouble(total, tests) :
    table = pd.DataFrame(columns = {'2', '3', '4', '5', '6', '7', '8', '9', '10', '11'})
    options = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
    option2 = 0
    option3 = 0
    option4 = 0
    option5 = 0
    option6 = 0
    option7 = 0
    option8 = 0
    option9 = 0
    option10 = 0
    option11 = 0
    for n in options :
        numResults = pd.DataFrame(columns = {'DubWin%: ', 'Win%: '})
        hitWins = 0
        hitPushes = 0
        standWins = 0
        standPushes = 0
        for num in range(tests) :
            playerHand = [2, total - 2]
            card = dealCard()
            if n == 11 :
                while card == 10 :
                    card = dealCard()
            dealerHand = [n, card]
            if not (handTotal(dealerHand) == 21) :
                playerHand = dealOneCard(playerHand)
                if didBust(playerHand) == False :
                    dealerHand = playDealer(dealerHand)
                    if didBust(dealerHand) | (handTotal(playerHand) > handTotal(dealerHand)) :
                        hitWins += 1
                    elif handTotal(playerHand) == handTotal(dealerHand) :
                         hitPushes += 1
        ph = hitWins / (tests - hitPushes)
        eh = (4 * ph) - 2
        numResults = numResults.append({'DubWin%: ' : eh, 'Win%: ': ph}, ignore_index = True)
        if n == 2:
            option2 = numResults
        elif n == 3:
            option3 = numResults
        elif n == 4:
            option4 = numResults
        elif n == 5:
            option5 = numResults
        elif n == 6:
            option6 = numResults
        elif n == 7:
            option7 = numResults
        elif n == 8:
            option8 = numResults
        elif n == 9:
            option9 = numResults
        elif n == 10:
            option10 = numResults
        elif n == 11:
            option11 = numResults
    table = table.append({'2' : option2, '3' : option3, '4' : option4, '5' : option5, '6' : option6, '7' : option7, '8' : option8, '9' : option9, '10' : option10, '11' : option11}, ignore_index = True)
    table = table[['2', '3', '4', '5', '6', '7', '8', '9', '10', '11']]
    return table

In [218]:
testChancesDouble(8, 100000)

,2,3,4,5,6,7,8,9,10,11
0,Win%: DubWin%: 0 0.44641 -0.214361,Win%: DubWin%: 0 0.460233 -0.159068,Win%: DubWin%: 0 0.482159 -0.071363,Win%: DubWin%: 0 0.501372 0.005489,Win%: DubWin%: 0 0.517332 0.069327,Win%: DubWin%: 0 0.446665 -0.213338,Win%: DubWin%: 0 0.370968 -0.516128,Win%: DubWin%: 0 0.306635 -0.773459,Win%: DubWin%: 0 0.278063 -0.88775,Win%: DubWin%: 0 0.353536 -0.585856


In [219]:
testChances(8, 100000)

,2,3,4,5,6,7,8,9,10,11
0,HitWin%: StayWin%: 0 -0.026739 -0.2832,HitWin%: StayWin%: 0 0.003199 -0.24748,HitWin%: StayWin%: 0 0.02971 -0.197,HitWin%: StayWin%: 0 0.074405 -0.16616,HitWin%: StayWin%: 0 0.110459 -0.11838,HitWin%: StayWin%: 0 0.088998 -0.47442,HitWin%: StayWin%: 0 -0.07048 -0.5133,HitWin%: StayWin%: 0 -0.233133 -0.54452,HitWin%: StayWin%: 0 -0.338546 -0.57544,HitWin%: StayWin%: 0 -0.188473 -0.44082


In [40]:
def createShoe(decks) :
    shoe = []
    for n in range(decks) :
        for num in range(4) :
            for number in range(2, 12) :
                shoe.append(number)
    return shoe

In [41]:
deck = createShoe(2)
deck

[2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11]